In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.show_dimensions", False)
pd.set_option("display.float_format", "{:4.2g}".format)

## 檔案的輸入輸出

### CSV檔案

> **LINK**

> http://air.epmap.org/

> 空氣質量資料來源：青悅空氣質量歷史資料庫

In [2]:
df_list = []

for df in pd.read_csv(
        u"data/aqi/上海市_201406.csv", 
        encoding="utf-8-sig",  #檔案解碼
        chunksize=100,         #一次讀入的行數
        usecols=[u"時間", u"監測點", "AQI", "PM2.5", "PM10"], #只讀入這些列
        na_values=["-", "—"],  #這些字串表示缺失資料
        parse_dates=[0]):      #第一列為時間列
    df_list.append(df)  #在這裡處理資料

%C df_list[0].count(); df_list[0].dtypes

df_list[0].count()     df_list[0].dtypes   
------------------  -----------------------
時間       100        時間       datetime64[ns]
監測點       90        監測點              object
AQI      100        AQI               int64
PM2.5    100        PM2.5             int64
PM10      98        PM10            float64
dtype: int64        dtype: object          


In [3]:
print type(df.loc[0, u"監測點"])

<type 'unicode'>


### HDF5檔案

> **LINK**

> http://www.nsmc.cma.gov.cn/FENGYUNCast/docs/HDF5.0_chinese.pdf

> 中文的HDF5使用簡介

In [5]:
store = pd.HDFStore("a.hdf5", complib="blosc", complevel=9)

In [6]:
df1 = pd.DataFrame(np.random.rand(100000, 4), columns=list("ABCD"))
df2 = pd.DataFrame(np.random.randint(0, 10000, (10000, 3)), 
                   columns=["One", "Two", "Three"])
s1 = pd.Series(np.random.rand(1000))
store["dataframes/df1"] = df1
store["dataframes/df2"] = df2
store["series/s1"] = s1
print store.keys()
print df1.equals(store["dataframes/df1"])

['/dataframes/df1', '/dataframes/df2', '/series/s1']
True


> **LINK**

> http://pytables.github.io/usersguide/libref/hierarchy_classes.html
>
> `pytables`官方文件

In [7]:
root = store.get_node("//")
for node in root._f_walknodes():
    print node

/dataframes (Group) u''
/series (Group) u''
/dataframes/df1 (Group) u''
/dataframes/df2 (Group) u''
/series/s1 (Group) u''
/series/s1/index (CArray(1000,), shuffle, blosc(9)) ''
/series/s1/values (CArray(1000,), shuffle, blosc(9)) ''
/dataframes/df1/axis0 (CArray(4,), shuffle, blosc(9)) ''
/dataframes/df1/axis1 (CArray(100000,), shuffle, blosc(9)) ''
/dataframes/df1/block0_items (CArray(4,), shuffle, blosc(9)) ''
/dataframes/df1/block0_values (CArray(100000, 4), shuffle, blosc(9)) ''
/dataframes/df2/axis0 (CArray(3,), shuffle, blosc(9)) ''
/dataframes/df2/axis1 (CArray(10000,), shuffle, blosc(9)) ''
/dataframes/df2/block0_items (CArray(3,), shuffle, blosc(9)) ''
/dataframes/df2/block0_values (CArray(10000, 3), shuffle, blosc(9)) ''


In [8]:
store.append('dataframes/df_dynamic1', df1, append=False) #❶
df3 = pd.DataFrame(np.random.rand(100, 4), columns=list("ABCD"))
store.append('dataframes/df_dynamic1', df3) #❷
store['dataframes/df_dynamic1'].shape

(100100, 4)

In [9]:
print store.select('dataframes/df_dynamic1', where='index > 97 & index < 102')

        A     B    C     D
98   0.95 0.072 0.78  0.18
99   0.19 0.043 0.24 0.075
100  0.21  0.78 0.86  0.47
101  0.71  0.87 0.63  0.74
98  0.058  0.18 0.91 0.083
99   0.47  0.81 0.71  0.59


In [10]:
store.append('dataframes/df_dynamic1', df1, append=False, data_columns=["A", "B"])
print store.select('dataframes/df_dynamic1', where='A > 0.99 & B < 0.01')

         A       B    C     D
3656  0.99  0.0018 0.67  0.47
5091     1   0.004 0.43  0.15
17671    1  0.0042 0.99  0.31
41052    1 0.00081  0.9  0.32
45307    1  0.0093 0.72 0.065
67976 0.99  0.0096 0.93  0.79
69078    1  0.0055 0.97  0.88
87871    1   0.008 0.59  0.35
94421 0.99  0.0049 0.36   0.9


> **WARNING**

> 由於所有從CSV檔案讀入`DataFrame`物件的行索引都為預設值，因此HDF5檔案中的資料的行索引並不是唯一的。

In [11]:
def read_aqi_files(fn_pattern):
    from glob import glob
    from os import path
    
    UTF8_BOM = b"\xEF\xBB\xBF"
    
    cols = "時間,城市,監測點,質量等級,AQI,PM2.5,PM10,CO,NO2,O3,SO2".split(",")
    float_dtypes = {col:float for col in "AQI,PM2.5,PM10,CO,NO2,O3,SO2".split(",")}
    names_map = {"時間":"Time", 
                 "監測點":"Position", 
                 "質量等級":"Level", 
                 "城市":"City", 
                 "PM2.5":"PM2_5"}
    
    for fn in glob(fn_pattern):
        with open(fn, "rb") as f:
            sig = f.read(3) #❶
            if sig != UTF8_BOM:
                f.seek(0, 0)
            df = pd.read_csv(f, 
                             parse_dates=[0], 
                             na_values=["-", "—"], 
                             usecols=cols, 
                             dtype=float_dtypes) #❷
        df.rename_axis(names_map, axis=1, inplace=True)  
        df.dropna(inplace=True)
        yield df


store = pd.HDFStore("data/aqi/aqi.hdf5", complib="blosc", complevel=9)
string_size = {"City": 12, "Position": 30, "Level":12}

for idx, df in enumerate(read_aqi_files(u"data/aqi/*.csv")):
    store.append('aqi', df, append=idx!=0, min_itemsize=string_size, data_columns=True) #❸
    
store.close()

In [12]:
store = pd.HDFStore("data/aqi/aqi.hdf5")
df_aqi = store.select("aqi")
print len(df_aqi)

337250


In [14]:
df_polluted = store.select("aqi", where="PM2_5 > 500")
print len(df_polluted)

87


### 讀寫資料庫

In [15]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data/aqi/aqi.db')

In [16]:
try:
    engine.execute("DROP TABLE aqi")
except:
    pass

In [17]:
str_cols = ["Position", "City", "Level"]

for df in read_aqi_files("data/aqi/*.csv"):
    for col in str_cols:
        df[col] = df[col].str.decode("utf8")
    df.to_sql("aqi", engine, if_exists="append", index=False)

In [18]:
df_aqi = pd.read_sql("aqi", engine)

In [19]:
df_polluted = pd.read_sql("select * from aqi where PM2_5 > 500", engine)
print len(df_polluted)

87


### 使用Pickle序列化

In [21]:
df_aqi.to_pickle("data/aqi/aqi.pickle")
df_aqi2 = pd.read_pickle("data/aqi/aqi.pickle")
df_aqi.equals(df_aqi2)

True